In [1]:
import pyodbc
server = 'localhost'
database = 'DBDWP511'
username = 'sa'
password = 'Numsey@Password!'
cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cursor = cnxn.cursor()

In [7]:
tsql = "select * from DimDate;"
with cursor.execute(tsql):
    for row in cursor:
        print(str(row[0:]))


    

/var/folders/9q/4j308r1x6y36n5_m3lmd7nrr0000gn/T/ipykernel_18206/2919149982.py:4: DeprecationWarning: PyUnicode_FromUnicode(NULL, size) is deprecated; use PyUnicode_New() instead
  print(str(row[0:]))


(20050101, datetime.date(2005, 1, 1), 7, 'Saturday', 'Sábado', 'Samedi', 1, 1, 1, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050102, datetime.date(2005, 1, 2), 1, 'Sunday', 'Domingo', 'Dimanche', 2, 2, 2, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050103, datetime.date(2005, 1, 3), 2, 'Monday', 'Lunes', 'Lundi', 3, 3, 2, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050104, datetime.date(2005, 1, 4), 3, 'Tuesday', 'Martes', 'Mardi', 4, 4, 2, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050105, datetime.date(2005, 1, 5), 4, 'Wednesday', 'Miércoles', 'Mercredi', 5, 5, 2, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050106, datetime.date(2005, 1, 6), 5, 'Thursday', 'Jueves', 'Jeudi', 6, 6, 2, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050107, datetime.date(2005, 1, 7), 6, 'Friday', 'Viernes', 'Vendredi', 7, 7, 2, 'January', 'Enero', 'Janvier', 1, 1, 2005, 1, 3, 2005, 2)
(20050108, datetime